# Task 1B: Fruit classification with a CNN

This notebook will serve as implementation of the API that you have created in your "Code" folder. You will write functions in the "py" files and use them here.

We will be using "Fruits" dataset present in PyTorch and train a convolutional neural network (CNN) to classify digits.


## What is expected from this notebook?

This notebook should be used to present your work. You should explain wherever necessary (but also not too much) about what you did and why you did it. You should explain things like hyper parameter settings (even if it was provided before hand to you by us), training performance and testing performance of the model. You should reason why your model is working fine and not overfitting.

Since numbers don't are an argot, you should also use visualizations wherever possible. You can visualize things like **loss curve**, show **confusion matrix** and since this is a CNN you can also consider **advance techniques like gradcam**, etc. 

You can also use techniques that allow for faster training, assuage problems involving vanishing and exploding gradients. 

Finally, you can show some manual verifications by displaying and making predictions on random test examples. 

## Absolutely required items?

1. First of all, import the libraries and the dataset. Divide the dataset into test and train.
2. Next, show dataset samples and distribution of different type of data. For example, in case of "Fruits Dataset" you can show some random images and their labels. Also, show distribution of each class of images.
3. Next, perform required transformations (also **data augmentation**) on "Fruits dataset" (normalization, resizing, grayscaling, if required, etc.) using torchvision transforms.
4. Create required dataloaders with PyTorch and use the module dataset we created to load data in mini-batches.
5. Train the model, show loss and accuracy at each step of operation.
6. Plot the **loss curve for both train and validation phase**.
7. Pick some manual random images (probably 7-10) from test dataset and predict their values showing **expected and actual result**. 

**NOTE: ** 
1. You may or may not choose to delete these instruction cells after completion of the notebook.
2. Keep the outputs of the cells.

In [1]:
from utils import dataset
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [2]:
# loading dataframes using dataset module 
df, df_train, df_test = dataset.create_and_load_meta_csv_df(dataset_path='../Data/fruits/', destination_path='../Data/', randomize=True, split=0.8)

In [3]:
# using dataframes, pytorch and torchvision to transform data. Also, use dataloaders for batching, shuffling, etc.  
data_transforms = {
    'train': [],
    'val': []
}

image_datasets = {'train': dataset.ImageDataset(df_train, transform=data_transforms['train']), 
                  'val': dataset.ImageDataset(df_test, transform=data_transforms['val'])}

dataloaders = {'train': DataLoader(image_datasets['train'],batch_size=20,shuffle=True,num_workers=2),
               'val' : DataLoader(image_datasets['val'],batch_size=20,shuffle=True,num_workers=2)}

In [4]:
import model

In [5]:
net = model.FNet()
net = net.to(device)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloaders['train'], 0):
    # get the inputs
        inputs, labels = data
        inputs = torch.transpose(inputs,3,1).float().to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 5 == 0:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f acc: %.3f' %
                  (epoch + 1, i + 1, running_loss / 5, 
                  (torch.sum(torch.argmax(outputs,1) == labels).float() / labels.size()[0]).item()))
            running_loss = 0.0

print('Finished Training')

[1,     1] loss: 0.321 acc: 0.150
[1,     6] loss: 1.613 acc: 0.200
[1,    11] loss: 1.635 acc: 0.150
[1,    16] loss: 1.609 acc: 0.200
[1,    21] loss: 1.610 acc: 0.150
[1,    26] loss: 1.594 acc: 0.100
[1,    31] loss: 1.579 acc: 0.100
[1,    36] loss: 1.602 acc: 0.400
[1,    41] loss: 1.584 acc: 0.300
[1,    46] loss: 1.576 acc: 0.200
[1,    51] loss: 1.593 acc: 0.500
[1,    56] loss: 1.582 acc: 0.400
[1,    61] loss: 1.561 acc: 0.500
[1,    66] loss: 1.516 acc: 0.300
[1,    71] loss: 1.515 acc: 0.500
[1,    76] loss: 1.517 acc: 0.500
[1,    81] loss: 1.470 acc: 0.200
[1,    86] loss: 1.433 acc: 0.250
[1,    91] loss: 1.380 acc: 0.500
[1,    96] loss: 1.341 acc: 0.650
[2,     1] loss: 0.264 acc: 0.550
[2,     6] loss: 1.313 acc: 0.350
[2,    11] loss: 1.441 acc: 0.400
[2,    16] loss: 1.184 acc: 0.550
[2,    21] loss: 1.266 acc: 0.600
[2,    26] loss: 1.284 acc: 0.550
[2,    31] loss: 1.222 acc: 0.300
[2,    36] loss: 1.268 acc: 0.600
[2,    41] loss: 1.330 acc: 0.400
[2,    46] los

In [10]:
accuracies = []
losses = []
epoch = 1
running_loss = 0.0
running_acc = 0.0
for i, data in enumerate(dataloaders['val'], 0):
    # get the inputs
    inputs, labels = data
    inputs = torch.transpose(inputs,3,1).float().to(device)
    labels = labels.to(device)
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    acc = torch.sum(torch.argmax(outputs,1) == labels).float() / labels.size()[0]
    accuracies.append(acc.item())
    losses.append(loss.item())
    running_loss += loss.item()
    running_acc += acc.item()
    if i % 5 == 0:    # print every 2000 mini-batches
        print('[%d, %5d] loss: %.3f acc: %.3f' %
              (epoch + 1, i + 1, running_loss / 5, running_acc / 5))
        running_loss = 0.0
        running_acc = 0.0

[2,     1] loss: 0.000 acc: 0.200
[2,     6] loss: 0.002 acc: 1.000
[2,    11] loss: 0.003 acc: 1.000
[2,    16] loss: 0.004 acc: 1.000
[2,    21] loss: 0.005 acc: 1.000
